In [80]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

In [87]:
"""
Gets all the coaches tags from basketball-reference.com. These tags include the URL for the coach's own page.
"""

tag_lst = []

for season in range(1997, 2021):
    page = requests.get("https://www.basketball-reference.com/leagues/NBA_{}_coaches.html".format(season))
    soup = BeautifulSoup(page.content, 'html.parser')
    for tag in soup.find_all('a'):
        if re.search('<a href="/coaches/', str(tag)) and re.search('html', str(tag)):
            tag_lst.append(tag)

# Remove duplicates            
tag_lst = list(dict.fromkeys(tag_lst))

In [155]:
"""
Creates a df with the coach's name and url to their page.
"""
coach_lst = []
i = 1
for tag in tag_lst:
    temp_lst = []
    temp_lst.append(i)
    i+=1
    for name in tag:
        temp_lst.append(name)
    temp_lst.append("https://www.basketball-reference.com" + str(tag).split('"')[1])
    
    coach_lst.append(temp_lst)
    
coach_df = pd.DataFrame(coach_lst, columns=['coach_id','coachName', 'url'])

In [157]:
"""
Collects the on the coaches
"""

df = pd.DataFrame(columns=['Season', 'Age', 'Tm', 'Lg', 'G', 'W', 'L', 'W/L%', 'W > .500',
       'Finish', 'G', 'W', 'L', 'W/L%', 'Notes', 'coachName', 'coach_id'])

for i, row in coach_df.iterrows():
    table = pd.io.html.read_html(row['url'])
    indexNames = list(dict.fromkeys([i[0] for i in table[0].columns]))
    temp_df = pd.DataFrame()
    for ind in indexNames:
        temp_df = pd.concat([temp_df, table[0][ind]], axis=1)
    temp_df['coachName'] = row.coachName
    temp_df['coach_id'] = row.coach_id
    temp_df.drop(columns=['Unnamed: 10_level_1'], inplace=True)
    
    df = pd.concat([df, temp_df])

In [173]:
"""
Cut values that do not include data on the season level
"""

def createMask(season):
    if len(str(season).split('-'))==2:
        return True
    else:
        return False
    
mask = df.Season.apply(createMask)

df = df[mask]

df.set_axis(['Age', 'Tm', 'Lg', 'G_reg', 'W_reg', 'L_reg', 'W/L%_reg', 'W > .500', 'Finish', 'G_playoff',
       'W_playoff', 'L_playoff', 'W/L%_playoff', 'Notes', 'coachName', 'coach_id', 'seasonStartYear'], axis = 1, inplace=True)

In [176]:
#Create season start year

df['seasonStartYear'] = df.Season.apply(lambda x: int(x.split('-')[0]))
df.drop(columns=['Season'], inplace=True)

C:\Users\Patrick\AppData\Local\Temp/ipykernel_23376/3102881218.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['seasonStartYear'] = df.Season.apply(lambda x: int(x.split('-')[0]))
C:\Users\Patrick\AppData\Local\Temp/ipykernel_23376/3102881218.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['Season'], inplace=True)


In [185]:
# Identify head coaches and clean the table

df['isHeadCoach'] = df.G_reg.apply(lambda x: 1 if x != 'Assistant Coach' else 0)

lst = []
isAssistantCoach = lambda x: x if x!= 'Assistant Coach' else '-'
for i, row in df.iterrows():
    if row.isHeadCoach == 1:
        lst.append(list(row))
    else:
        lst.append([isAssistantCoach(i) for i in list(row)])
        
df_clean = pd.DataFrame(lst, columns=list(df.columns))

df_clean = df_clean.fillna('-')

C:\Users\Patrick\AppData\Local\Temp/ipykernel_23376/1196380395.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['isHeadCoach'] = df.G_reg.apply(lambda x: 1 if x != 'Assistant Coach' else 0)


In [202]:
df_clean.to_csv("coaches.csv", index=False)